<a href="https://colab.research.google.com/github/eli-vigneron/misc_projects/blob/main/lightfm_reccomendation_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
features = pd.read_csv('/content/drive/MyDrive/features.csv')
ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv')

In [ ]:
features.head()

,product_id,product_class_id,product_year,product_name,country_name,feature1,feature2,feature3,feature4,feature5,num_ratings,avg_ratings,region_name,food1,food2,food3,food4,food5,food6,food7,note1,note2,note3,note4,note5,note6,note7,note8,note9,note10,note11,note12,note13,class_name,price
0,724939662765,10084382031,10366272,1,10000.0,11480.168232,18508.557202,11210.750328,13386.629203,0.0,89979024,7.4,15000.0,1,1,0,0,0,0,1,66009.78262062,2129.347826598,8517.391302474,53233.69564536,4258.695653196,21293.478262062,2129.347826598,112855.43479794,95820.65217732,12776.08695567,0,0,12776.08695567,20000,19.95
1,1347498441408,3287810067,9543813,2,10001.0,21150.199920,12395.008618,6815.636914,0.000000,0.0,9592056,7.6,15001.0,0,0,0,0,0,0,0,3957.57575718,122684.84847258,3957.57575718,67278.78787206,3957.57575718,3957.57575718,7915.15151436,7915.15151436,3957.57575718,3957.57575718,91024.24241514,23745.454546998,47490.90908616,20001,17.95
2,1022923631932,28349197767,12967668,3,10002.0,19865.256912,10852.594147,4928.222765,0.000000,0.0,830808,7.6,15002.0,0,0,0,0,0,1,0,0,180830.7692157,0,60276.92305884,0,0,0,0,0,30138.461537256,60276.92305884,30138.461537256,30138.461537256,20002,14.95
3,5182824340,9062972469,92275236,4,10003.0,18501.937258,20850.229920,7673.151638,19155.065150,0.0,8979440,7.0,15003.0,1,0,1,0,0,1,0,55971.42858822,9328.571428758,9328.571428758,27985.714286274,0,4664.28571242,4664.28571242,149257.1428758,79292.85715032,46642.8571242,0,0,4664.28571242,20003,12.00
4,956452733050,21064890288,12967668,5,10002.0,0.000000,0.000000,0.000000,0.000000,0.0,134272,7.6,15004.0,0,0,0,0,0,0,0,0,0,0,97950,0,97950,0,0,0,0,97950,97950,0,20004,17.20


In [ ]:
ratings.head()

,user_id,product_id,rating
0,8765713110,1.623200e+12,10
1,8765713110,1.623200e+12,10
2,8765713110,1.623200e+12,10
3,8765713110,1.623200e+12,10
4,8765713110,1.623200e+12,10


In [ ]:
ratings.isnull().sum()

user_id        0
product_id    33
rating         0
dtype: int64

In [ ]:
# ratings.dropna(inplace=True)
ratings['product_id'] = ratings['product_id'].fillna('No ID')

In [ ]:
features.isnull().sum()

product_id             0
product_class_id       0
product_year           0
product_name           0
country_name          65
feature1               0
feature2               0
feature3               0
feature4               0
feature5               0
num_ratings            0
avg_ratings            0
region_name         1604
food1                  0
food2                  0
food3                  0
food4                  0
food5                  0
food6                  0
food7                  0
note1                  0
note2                  0
note3                  0
note4                  0
note5                  0
note6                  0
note7                  0
note8                  0
note9                  0
note10                 0
note11                 0
note12                 0
note13                 0
class_name             0
price                  0
dtype: int64

In [ ]:
features.fillna(0, inplace=True)
# features.dropna(inplace=True)

In [ ]:
!pip install lightfm

In [ ]:
from lightfm.data import Dataset

# create lightfm internel mapping
dataset = Dataset()
dataset.fit(ratings['user_id'].unique(),
            features['product_id'].unique(),
            user_features=ratings.values.flatten(),
            item_features=features.values.flatten()
            )

In [ ]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 16836, num_items 3913.


In [ ]:
dataset.fit_partial(items=(row['product_id'] for index, row in features.iterrows()),
                    item_features=features.loc[:, 'product_class_id':'price'].values.flatten())
                    #item_features=features[['feature1','feature2','feature3','feature4']].values.flatten())

In [ ]:
(interactions, weights) = dataset.build_interactions(((row['user_id'], row['product_id']) for index, row in ratings.iterrows()))

print(repr(interactions))

<16836x3913 sparse matrix of type '<class 'numpy.int32'>'
	with 93296 stored elements in COOrdinate format>


In [ ]:
item_features = dataset.build_item_features(((row['product_id'], 
                                              #row['product_class_id':'price'])
                                              row[['product_class_id', 'feature1','feature2','feature3','feature4']])
                                              for index, row in features.iterrows()))
#item_features = dataset.build_item_features(features[['product_id', 'feature1','feature2','feature3','feature4']].values.flatten())
print(repr(item_features))

<4396x38722 sparse matrix of type '<class 'numpy.float32'>'
	with 23425 stored elements in Compressed Sparse Row format>


In [ ]:
from lightfm import LightFM
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split

train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=None)

# Set the number of threads; you can increase this
# if you have more physical cores available.
NUM_THREADS = 4
NUM_COMPONENTS = 30
NUM_EPOCHS = 80
ITEM_ALPHA = 1e-6

In [ ]:
# Let's fit a WARP model: these generally have the best performance.
model = LightFM(loss='warp',
                item_alpha=ITEM_ALPHA)

# Run 3 epochs and time it.
%time model = model.fit(train, epochs=NUM_EPOCHS, num_threads=NUM_THREADS)

CPU times: user 8.92 s, sys: 24 ms, total: 8.95 s
Wall time: 4.92 s


In [ ]:
# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute and print the train AUC score
train_auc = auc_score(model, train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

Collaborative filtering train AUC: 0.9851957


In [ ]:
# We pass in the train interactions to exclude them from predictions.
# This is to simulate a recommender system where we do not
# re-recommend things the user has already interacted with in the train
# set.
test_auc = auc_score(model, test, num_threads=NUM_THREADS).mean()
print('Collaborative filtering test AUC: %s' % test_auc)

Collaborative filtering test AUC: 0.8134483


In [ ]:
# Define a new model instance
model = LightFM(loss='warp',
                item_alpha=ITEM_ALPHA,
                no_components=NUM_COMPONENTS)

# Fit the hybrid model. Note that this time, we pass
# in the item features matrix.
model = model.fit(train,
                item_features=item_features,
                epochs=NUM_EPOCHS,
                num_threads=NUM_THREADS)

# Don't forget the pass in the item features again!
train_auc = auc_score(model,
                      train,
                      item_features=item_features,
                      num_threads=NUM_THREADS).mean()
print('Hybrid training set AUC: %s' % train_auc)

test_auc = auc_score(model,
                    test,
                    #train_interactions=train, # Unable to run with this line uncommented
                    item_features=item_features,
                    num_threads=NUM_THREADS).mean()
print('Hybrid test set AUC: %s' % test_auc)

Hybrid training set AUC: 0.8884649
Hybrid test set AUC: 0.81616676
